# Classes base abstratas e métodos abstratos

Em algumas situações, queremos definir um método em uma classe base que não tem nenhuma implementação específica nessa classe, mas terá apenas implementação válida nas classes derivadas.

Como um exemplo arbitrário, o código abaixo define um método `h` que faz uso de um método `m` que não foi definido.

In [ ]:
class Base:
    def __init__(self):
        print('Creating a Base')
    def f(self):
        print('f of Base')
    def h(self):
        self.m() # Chamada de método não definido!

In [ ]:
class Derived1(Base):
    def g(self):
        print('g of Derived1')

In [ ]:
class Derived2(Base):
    def f(self):
        print('f of Derived2')

In [ ]:
class Derived3(Base):
    def f(self):
        Base.f(self)
        print('f of Derived3')

In [ ]:
class Derived4(Base):
    def m(self):
        print('m of Derived4')

Das classes acima, apenas a `Derived4` é completa, pois nenhuma das outras define `m`, e portanto não podemos executar `h` sobre objetos dessas classes.

In [ ]:
b = Base()

In [ ]:
b.f()

In [ ]:
d1 = Derived1()

In [ ]:
d1.f()

In [ ]:
d1.g()

In [ ]:
d2 = Derived2()

In [ ]:
d2.f()

In [ ]:
d3 = Derived3()

In [ ]:
d3.f()

In [ ]:
d1.h()

In [ ]:
d2.h()

In [ ]:
d3.h()

In [ ]:
d4 = Derived4()

In [ ]:
d4.f()

In [ ]:
d4.h()

Dizemos nesse caso que a classe base é **abstrata** e o método `m` é um **método abstrato**.

O ideal é indicar isso claramente no código, pois dessa forma o Python pode impedir que criemos objetos incompletos. Isso é feito com auxílio do módulo `abc` (de **a**bstract **b**ase **c**lass).

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class Base(ABC):
    def __init__(self):
        print('Creating a Base')
    def f(self):
        print('f of Base')
    def h(self):
        self.m()
    @abstractmethod
    def m(self):
        pass

O uso de `ABC` na lista de classes base indica que a classe definida é uma classe base abstrata.

O uso de `@abstractmethod` antes da definição do método `m` indica que esse é um método abstrato. A notação com o `@` indica que `abstractmethod` é um **decorador**. Estudaremos decoradores mais para frente.

Com essa definição, não é possível criar objetos da classe `Base`, pois ela está registrada no Python como abstrata.

In [ ]:
b = Base()

O mesmo vale para qualquer classe derivada que não implementar o método `m`.

In [ ]:
class Derived1(Base):
    def g(self):
        print('g de Derivada1')

In [ ]:
d1 = Derived1()

Para criar objetos, precisamos definir uma classe derivada que implementa o método abstrato `m`.

In [ ]:
class Derived4(Base):
    def m(self):
        print('m of Derived4')

In [ ]:
d4 = Derived4()

In [ ]:
d4.h()

## Exemplo

Como exemplo, vamos agora considerar um caso excessivamente simplificado de cálculo do salário líquido a ser pago em uma empresa.

Nessa empresa simplificada desse país simplificado, temos três tipos de colaboradores: escriturários, vendedores e gerentes. Os escriturários recebem um salário bruto fixo, os vendedores, além de uma base fixa, recebem uma comissão de 10% sobre o total de vendas realizadas por eles, por fim, os gerentes, além de uma base fixa, recebem uma comissão de 5% sobre o lucro líquido do seu departamento.

Por outro lado, do valor do salário bruto são descontados 10% para segurança social e do restante são descontados 15% de imposto de renda na fonte.

Para os cálculos, os dados sobre cada colaborador são fornecidos em um arquivo (aqui usaremos apenas uma cadeia de caracteres fixa, para simplificar) com o seguinte formato:
- Cada linha corresponde a um colaborador
- Cada linha é constituída de campos separados por vírgulas:
  - O primeiro campo é o nome do colaborador
  - O segundo campo é o seu cargo (manager, sales ou clerk)
  - O terceiro campo é o salário base
  - No caso de um vendedor, há um quarto campo com o total de vendas que ele realizou
  - No caso de um gerente, há um quarto campo com o lucro líquido do seu departamento
  
Primeiro, vamos definir uma classe base com o comportamento comum a todos os tipos de colaboradores:

In [ ]:
class Employee(ABC):
    TAX_RATE = 0.15
    SOCIAL_SECURITY_RATE = 0.1
    
    def __init__(self, name):
        self._name = name
        self._base_salary = None
    
    def get_name(self):
        return self._name
    
    def set_salary(self, salary):
        self._base_salary = salary
    
    @abstractmethod
    def compute_gross(self):
        pass
    
    def net_salary(self):
        base = self.compute_gross() * (1 - Employee.SOCIAL_SECURITY_RATE)
        net = base * (1 - Employee.TAX_RATE)
        return net

Agora definimos uma classe para cada um dos tipos específicos de colaborador. Com o que já temos implementado na classe `Employee`, basta implementar o método abstrato `compute_gross` apropriado para cada uma das categorias.

Um escriturário recebe apenas o salário base.

In [ ]:
class Clerk(Employee):
    def compute_gross(self):
        return self._base_salary   

Um vendedor recebe adicionalmente uma comissão de vendas. Precisamos então poder registrar o total de vendas, e para isso acrescentamos o método `set_sales`, além de implementar `compute_gross`. Para a implementação de `compute_gross` precisamos do valor da porcentagem da comissão de venda. Para deixar o código mais fácil de alterar, definimos a constante `COMMMISSION_RATE`; como essa constante só é de interesse para essa classe, a definimos localmente (e não como uma constante global); como ela é igual para todos os vendedores, a definimos como membro da classe (e não do objeto).

In [ ]:
class Salesperson(Employee):
    COMMISSION_RATE = 0.1
    
    def __init__(self, name = None):
        Employee.__init__(self, name)
        self._sales = None
    
    def set_sales(self, sales_volume):
        self._sales = sales_volume

    def compute_gross(self):
        return self._base_salary + self._sales * Salesperson.COMMISSION_RATE

De forma similar, definimos para os gerente um método para guardar o valor do lucro do departamento e uma constante de classe com a taxa de comissão neste caso.

In [ ]:
class Manager(Employee):
    COMMISSION_RATE = 0.05
    
    def __init__(self, name = None):
        Employee.__init__(self, name)
        self._dept_profit = None
        
    def set_profit(self, net_profit):
        self._dept_profit = net_profit
        
    def compute_gross(self):
        return self._base_salary + self._dept_profit * Manager.COMMISSION_RATE

Note como tanto no `Salesperson` como no `Manager`, criamos o campo adicional necessário no método `__init__`, apesar de neste momento ainda não sabermos o valor (inicializamos com `None` por essa razão). Como já comentado, isso não é necessário, pois qualquer método pode a qualquer momento criar novos membros nos objetos, mas da forma que foi feito, se for feita uma chamada de `compute_gross` para esses objetos antes dos campos necessários serem ajustados, teremos um erro indicando que o valor é `None`, ao invés de um erro indicando que o membro apropriado não existe, o que me parece uma mensagem mais apropriada (`None` indica algo desconhecido).

Agora vamos inicializar nossa base de dados:

In [ ]:
EMPLOYEE_DATABASE = '''
Carlos Pedrosa, manager, 10000, 1000000
Paola Teixeira, sales, 2000, 30000
José Prado, sales, 2000, 27500
Tadeu Costa, sales, 2200, 20000
Victor Duarte, clerk, 1500
Angelo Rodrigues, clerk, 1700
Marta Cardoso, clerk, 1800
'''

Basta agora percorrer cada linha da cadeia, separar os campos da linha por vírgula, criar os objetos apropriados a cada linha, e inicializar os membros correspondentes do objeto.

In [ ]:
employees = []
for line in EMPLOYEE_DATABASE.strip().split('\n'):
    fields = [field.strip() for field in line.split(',')]
    name = fields[0]
    position = fields[1]
    salary = float(fields[2])
    if position == 'manager':
        employee = Manager(name)
        profit = float(fields[3])
        employee.set_profit(profit)
    elif position == 'sales':
        employee = Salesperson(name)
        sales = float(fields[3])
        employee.set_sales(sales)
    elif position == 'clerk':
        employee = Clerk(name)
    else:
        raise RuntimeError(f'{position} is not a valid job description')
    employee.set_salary(salary)
    employees.append(employee)

Estamos agora prontos para verificar os valores a pagar a cada colaborador:

In [ ]:
for e in employees:
    print(e.get_name(),'receives', e.net_salary())

In [ ]:
employees

# Exercícios

1. De quais classes abaixo podemos criar objetos e de quais não?
```python
from abc import ABC, abstractmethod
class A:
    def f(self):
        print('From A')

class B(A):
    pass

class C(ABC):
    def f(self):
        print('From C')
    @abstractmethod
    def g(self):
        pass

class D(C):
    def f(self):
        print('From D')

class E(C):
    def g(self):
        print('Got E')

```

2. O código abaixo foi escrito por um programador que não conhece o conceito de classes base abstratas. Reescreva esse código com a ajuda do módulo `abc`.
```python
import math
class Solid:
    def __init__(self, mass):
        self._mass = mass
    def mass(self):
        return self._mass
    def volume(self):
        pass
    def density(self):
        return self._mass / self.volume()

class RectangularParallelepiped(Solid):
    def __init__(self, mass, lx, ly, lz):
        Solid.__init__(self, mass)
        self._x = lx
        self._y = ly
        self._z = lz
    def volume(self):
        return self._x * self._y * self._z

class Sphere(Solid):
    def __init__(self, mass, r):
        Solid.__init__(self, mass)
        self._r = r
    def volume(self):
        return 4 / 3 * math.pi * self._r ** 3
```